# Data Preprocessing - Census Income Prediction

This notebook performs data preprocessing steps:
- Remove duplicate rows
- Fill null values
- Treat outliers
- Make label binary (0, 1)
- Optional: Encode categoricals (one-hot encoding)
- Feature engineering
- Split into train, validation, and test sets


In [1]:
# Import necessary libraries
import sys
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path.cwd().parent))

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Import project modules
from src.data_loading import load_data, get_column_names
from src.data_preprocessing import (
    remove_duplicates,
    handle_missing_values,
    treat_outliers,
    make_label_binary,
    encode_categorical,
    feature_engineering,
    split_train_val_test
)

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

print("Libraries imported successfully!")


Libraries imported successfully!


## 1. Load Data


In [2]:
# Load data
train_path = "../data/census_income_learn.csv"
test_path = "../data/census_income_test.csv"

print("Loading data...")
train_df, test_df = load_data(train_path, test_path)

# Get column names
columns = get_column_names()
train_df.columns = columns
test_df.columns = columns

print(f"Training data shape: {train_df.shape}")
print(f"Test data shape: {test_df.shape}")
print(f"\nTarget column: {columns[-1]}")

# Display first few rows
train_df.head()


Loading data...
Training data shape: (199523, 42)
Test data shape: (99762, 42)

Target column: income


,age,class_of_worker,detailed_industry_recode,detailed_occupation_recode,education,wage_per_hour,enroll_in_edu_inst_last_wk,marital_stat,major_industry_code,major_occupation_code,race,hispanic_origin,sex,member_of_labor_union,reason_for_unemployment,full_or_part_time_employment_stat,capital_gains,capital_losses,dividends_from_stocks,tax_filer_stat,region_of_previous_residence,state_of_previous_residence,detailed_household_and_family_stat,detailed_household_summary_in_household,instance_weight,migration_code_change_in_msa,migration_code_change_in_reg,migration_code_move_within_reg,live_in_this_house_1_year_ago,migration_prev_res_in_sunbelt,num_persons_worked_for_employer,family_members_under_18,country_of_birth_father,country_of_birth_mother,country_of_birth_self,citizenship,own_business_or_self_employed,fill_inc_questionnaire_for_veterans_admin,veterans_benefits,weeks_worked_in_year,year,income
0,73,Not in universe,0,0,High school graduate,0,Not in universe,Widowed,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Not in labor force,0,0,0,Nonfiler,Not in universe,Not in universe,Other Rel 18+ ever marr not in subfamily,Other relative of householder,1700.09,?,?,?,Not in universe under 1 year old,?,0,Not in universe,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,0,95,- 50000.
1,58,Self-employed-not incorporated,4,34,Some college but no degree,0,Not in universe,Divorced,Construction,Precision production craft & repair,White,All other,Male,Not in universe,Not in universe,Children or Armed Forces,0,0,0,Head of household,South,Arkansas,Householder,Householder,1053.55,MSA to MSA,Same county,Same county,No,Yes,1,Not in universe,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,52,94,- 50000.
2,18,Not in universe,0,0,10th grade,0,High school,Never married,Not in universe or children,Not in universe,Asian or Pacific Islander,All other,Female,Not in universe,Not in universe,Not in labor force,0,0,0,Nonfiler,Not in universe,Not in universe,Child 18+ never marr Not in a subfamily,Child 18 or older,991.95,?,?,?,Not in universe under 1 year old,?,0,Not in universe,Vietnam,Vietnam,Vietnam,Foreign born- Not a citizen of U S,0,Not in universe,2,0,95,- 50000.
3,9,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Children or Armed Forces,0,0,0,Nonfiler,Not in universe,Not in universe,Child <18 never marr not in subfamily,Child under 18 never married,1758.14,Nonmover,Nonmover,Nonmover,Yes,Not in universe,0,Both parents present,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,0,0,94,- 50000.
4,10,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Children or Armed Forces,0,0,0,Nonfiler,Not in universe,Not in universe,Child <18 never marr not in subfamily,Child under 18 never married,1069.16,Nonmover,Nonmover,Nonmover,Yes,Not in universe,0,Both parents present,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,0,0,94,- 50000.


## 2. Remove Duplicate Rows


In [3]:
print("=" * 80)
print("REMOVE DUPLICATE ROWS")
print("=" * 80)

# Check duplicates before removal
target_col = 'income'
print(f"\nBefore removal:")
print(f"Training data shape: {train_df.shape}")
print(f"Test data shape: {test_df.shape}")

# Remove duplicates excluding instance_weight (survey weights may cause legitimate duplicates)
# Note: See EDA notebook Section 4 for detailed explanation of why instance_weight causes duplicates
columns_without_weight = [col for col in train_df.columns if col != 'instance_weight']

train_df = remove_duplicates(train_df, subset=columns_without_weight, keep='first')
test_df = remove_duplicates(test_df, subset=columns_without_weight, keep='first')

print(f"\nAfter removal:")
print(f"Training data shape: {train_df.shape}")
print(f"Test data shape: {test_df.shape}")


REMOVE DUPLICATE ROWS

Before removal:
Training data shape: (199523, 42)
Test data shape: (99762, 42)

After removal:
Training data shape: (152896, 42)
Test data shape: (78864, 42)


## 3. Fill Null Values


In [4]:
print("=" * 80)
print("FILL NULL VALUES")
print("=" * 80)

# Handle missing values (replace '?' with NaN and fill)
# Categorical columns: fill with "not identified"
# Numerical columns: fill with median (or specify numerical_fill_value, e.g., -1)
train_df = handle_missing_values(
    train_df,
    strategy='mode',  # Fill categorical with "not identified", numerical with median
    missing_indicators=["?", " ?", "? ", " ? "],
    categorical_fill_value="not identified",
    numerical_fill_value=None  # None = use median, or specify a value like -1
)

test_df = handle_missing_values(
    test_df,
    strategy='mode',
    missing_indicators=["?", " ?", "? ", " ? "],
    categorical_fill_value="not identified",
    numerical_fill_value=None  # None = use median, or specify a value like -1
)

# Check remaining nulls
print(f"\nRemaining nulls in training data: {train_df.isnull().sum().sum()}")
print(f"Remaining nulls in test data: {test_df.isnull().sum().sum()}")

if train_df.isnull().sum().sum() > 0:
    print("\nColumns with nulls in training data:")
    print(train_df.isnull().sum()[train_df.isnull().sum() > 0])

if test_df.isnull().sum().sum() > 0:
    print("\nColumns with nulls in test data:")
    print(test_df.isnull().sum()[test_df.isnull().sum() > 0])


FILL NULL VALUES

Remaining nulls in training data: 0
Remaining nulls in test data: 0


## 4. Treat Outliers


In [5]:
print("=" * 80)
print("TREAT OUTLIERS")
print("=" * 80)

# Identify numerical columns (excluding target and instance_weight)
numerical_cols = train_df.select_dtypes(include=[np.number]).columns.tolist()
numerical_cols = [col for col in numerical_cols if col not in [target_col, 'instance_weight']]

print(f"\nNumerical columns to treat: {len(numerical_cols)}")
print(f"Columns: {numerical_cols[:10]}..." if len(numerical_cols) > 10 else f"Columns: {numerical_cols}")

# Treat outliers using winsorization (clip to 1st and 99th percentiles)
# Calculate bounds from training data, then apply same bounds to test data
train_df, outlier_bounds = treat_outliers(
    train_df,
    columns=numerical_cols,
    method='winsorize',
    lower_percentile=0.01,
    upper_percentile=0.99
)

# Apply same bounds from training data to test data
test_df, _ = treat_outliers(
    test_df,
    columns=numerical_cols,
    method='winsorize',
    bounds=outlier_bounds
)

print("\nOutliers treated using winsorization (1st-99th percentiles)")
print("Bounds calculated from training data and applied to both train and test")


TREAT OUTLIERS

Numerical columns to treat: 12
Columns: ['age', 'detailed_industry_recode', 'detailed_occupation_recode', 'wage_per_hour', 'capital_gains', 'capital_losses', 'dividends_from_stocks', 'num_persons_worked_for_employer', 'own_business_or_self_employed', 'veterans_benefits']...

Outliers treated using winsorization (1st-99th percentiles)
Bounds calculated from training data and applied to both train and test


In [6]:
train_df[target_col].drop_duplicates()

0      - 50000.
57      50000+.
Name: income, dtype: object

## 5. Make Label Binary


In [7]:
print("=" * 80)
print("MAKE LABEL BINARY")
print("=" * 80)

# Check current target distribution
print(f"\nTraining target distribution:")
print(train_df[target_col].value_counts())
print(f"\nTest target distribution:")
print(test_df[target_col].value_counts())

# Convert to binary (50000+. -> 1, -50000. -> 0)
train_df = make_label_binary(train_df, target_column=target_col, positive_class='50000+.')
test_df = make_label_binary(test_df, target_column=target_col, positive_class='50000+.')

# Check binary target distribution
print(f"\nAfter binary conversion:")
print(f"Training target distribution:")
print(train_df[target_col].value_counts())
print(f"\nTest target distribution:")
print(test_df[target_col].value_counts())


MAKE LABEL BINARY

Training target distribution:
income
- 50000.    140529
50000+.      12367
Name: count, dtype: int64

Test target distribution:
income
- 50000.    72678
50000+.      6186
Name: count, dtype: int64

After binary conversion:
Training target distribution:
income
0    140529
1     12367
Name: count, dtype: int64

Test target distribution:
income
0    72678
1     6186
Name: count, dtype: int64


## 6. Feature Engineering


In [8]:
print("=" * 80)
print("FEATURE ENGINEERING")
print("=" * 80)

# Apply feature engineering
train_df = feature_engineering(train_df, target_column=target_col)
test_df = feature_engineering(test_df, target_column=target_col)

print(f"\nAfter feature engineering:")
print(f"Training data shape: {train_df.shape}")
print(f"Test data shape: {test_df.shape}")

# Show new features created
new_features = [col for col in train_df.columns if col not in columns]
if new_features:
    print(f"\nNew features created: {len(new_features)}")
    print(f"Features: {new_features}")


FEATURE ENGINEERING

After feature engineering:
Training data shape: (152896, 49)
Test data shape: (78864, 49)

New features created: 7
Features: ['has_capital_gains', 'has_capital_losses', 'has_dividends', 'has_wage', 'age_group', 'total_financial_assets', 'work_intensity']


## 7. Encode Categoricals (One-Hot Encoding)


In [9]:
if False:
    print("=" * 80)
    print("ENCODE CATEGORICALS (ONE-HOT ENCODING)")
    print("=" * 80)
    
    # Identify categorical columns (excluding target)
    categorical_cols = train_df.select_dtypes(include=['object']).columns.tolist()
    categorical_cols = [col for col in categorical_cols if col != target_col]
    
    print(f"\nCategorical columns to encode: {len(categorical_cols)}")
    print(f"Columns: {categorical_cols[:10]}..." if len(categorical_cols) > 10 else f"Columns: {categorical_cols}")
    
    # Apply one-hot encoding to training data
    train_df_encoded, _ = encode_categorical(
        train_df,
        method='onehot',
        columns=categorical_cols
    )
    
    # Apply one-hot encoding to test data
    # Note: Test data might have different categories, so we need to align columns
    test_df_encoded, _ = encode_categorical(
        test_df,
        method='onehot',
        columns=categorical_cols
    )
    
    # Align columns (add missing columns with zeros, remove extra columns)
    train_cols = set(train_df_encoded.columns)
    test_cols = set(test_df_encoded.columns)
    
    # Add missing columns to test (fill with 0)
    missing_in_test = train_cols - test_cols
    for col in missing_in_test:
        if col != target_col:
            test_df_encoded[col] = 0
    
    # Remove extra columns from test
    extra_in_test = test_cols - train_cols
    for col in extra_in_test:
        if col != target_col:
            test_df_encoded = test_df_encoded.drop(columns=[col])
    
    # Ensure same column order
    test_df_encoded = test_df_encoded[train_df_encoded.columns]
else:
    train_df_encoded = train_df
    test_df_encoded = test_df
    
print(f"\nAfter encoding:")
print(f"Training data shape: {train_df_encoded.shape}")
print(f"Test data shape: {test_df_encoded.shape}")
print(f"\nTotal features: {len(train_df_encoded.columns) - 1}")  # Excluding target



After encoding:
Training data shape: (152896, 49)
Test data shape: (78864, 49)

Total features: 48


## 8. Split into Train, Validation, and Test Sets


In [10]:
print("=" * 80)
print("SPLIT INTO TRAIN, VALIDATION, AND TEST SETS, the original learn csv is split while the test file is maintained")
print("=" * 80)

# Split data
X_train, X_val, X_test, y_train, y_val, y_test = split_train_val_test(
    train_df_encoded,
    test_df_encoded,
    target_column=target_col,
    val_size=0.2,
    random_state=42
)

print(f"\nData splits:")
print(f"Training set: {X_train.shape[0]} samples, {X_train.shape[1]} features")
print(f"Validation set: {X_val.shape[0]} samples, {X_val.shape[1]} features")
print(f"Test set: {X_test.shape[0]} samples, {X_test.shape[1]} features")

print(f"\nTarget distribution:")
print(f"Training: {y_train.value_counts().to_dict()}")
print(f"Validation: {y_val.value_counts().to_dict()}")
print(f"Test: {y_test.value_counts().to_dict()}")


SPLIT INTO TRAIN, VALIDATION, AND TEST SETS, the original learn csv is split while the test file is maintained

Data splits:
Training set: 122316 samples, 48 features
Validation set: 30580 samples, 48 features
Test set: 78864 samples, 48 features

Target distribution:
Training: {0: 112422, 1: 9894}
Validation: {0: 28107, 1: 2473}
Test: {0: 72678, 1: 6186}


## 9. Save Processed Data


In [11]:
print("=" * 80)
print("SAVE PROCESSED DATA")
print("=" * 80)

# Create output directory
output_dir = Path("../data/processed")
output_dir.mkdir(parents=True, exist_ok=True)

# Combine features and targets for saving
train_processed = pd.concat([X_train, y_train], axis=1)
val_processed = pd.concat([X_val, y_val], axis=1)
test_processed = pd.concat([X_test, y_test], axis=1)

# Save to CSV
train_path = output_dir / "train_processed.csv"
val_path = output_dir / "val_processed.csv"
test_path = output_dir / "test_processed.csv"

train_processed.to_csv(train_path, index=False)
val_processed.to_csv(val_path, index=False)
test_processed.to_csv(test_path, index=False)

print(f"\nProcessed data saved to:")
print(f"  - {train_path}")
print(f"  - {val_path}")
print(f"  - {test_path}")

print(f"\nPreprocessing complete!")


SAVE PROCESSED DATA

Processed data saved to:
  - ../data/processed/train_processed.csv
  - ../data/processed/val_processed.csv
  - ../data/processed/test_processed.csv

Preprocessing complete!
